In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import flask
from flask import Flask, request, jsonify
import config
from db_connect import db

import pandas as pd
import numpy as np
import torch
import tensorflow as tf


#bring AI model
model = tf.keras.models.load_model('model path')
model.eval()


@app.route('/')
def home():
   return 'This is Home!'


#
#This code will be changed by the rule.
#if we just use spring server to get sign of data store, then get data from bucket or db.
#

#혹시 모르니 request 받을 용도 미완!
@app.route('/fromSpring',methods = ["POST"]):
    response = {'Success': False}
    params = request.json  #maybe json..?
    
    if params == None:
        params = request.args

    return jsonify(reponse.json()), 200




#communicate with spring server
@app.route('/sendToSpring', methods= ["POST"])
def send_to_spring():
  
    datas = get_data_from_rds() #get data from aws rds


    data_array = []

    for line in datas:
      data = data_transfrom(line['data'])   #change data format to use 
      result = model_predict(x)

      if result != 0:
          resp = {'index':, 'result': }
          resp['index'] = line['index']
          resp['result'] = result

          data_array.append(resp)


    send_data = json.dumps(data_array,ident = 2) #dictionary array to json



    #send to spring server
    response = requests.post('http://spring_server_url:port/receive_data', json=send_data)
    # 응답 처리 코드
    return jsonify(response.json()), 200



def model_predict(data):
    #아직 어떤 처리를 해야할지 모르겠음.
    return model(data)


    
#if get data file from Spring. it makes data useful to model
def data_transform(data):
    df = pd.DataFrame.from_dict(data) # need to consider index
    return torch.tensor(df)




#make connection with AWS RDS DB
def create_app(test_config = None):
    app = Flask(__name__)

    if test_config = None:
      app.config.from_object(config)
    else:
      app.config.update(test_config)

    db.init_app(app)


def get_data_from_rds():
    cursor = conn.cursor(pymysql.cursors.DictCursor) #dictionary 형태로 읽어오기

    # 쿼리문 실행
    cursor.execute("SELECT * FROM table_name")
    result = cursor.fetchall()

    # 데이터 처리 및 응답 생성
    data = []
    for row in result:
        data.append({'index': row['index'], 'data': row['data']})
    return jsonify(data), 200



if __name__ == '__main__':

    create_app().run('0.0.0.0',port=8000,debug=True)